In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import os
from pathlib import Path
from PIL import Image

def make_white_transparent(image_path, output_path=None, white_threshold=240):
    """
    Beyaz pikselleri şeffaf yapar (alpha=0)

    Parameters:
    -----------
    image_path : str
        Giriş görüntüsü
    output_path : str
        Çıkış yolu (PNG olarak kaydedilir)
    white_threshold : int
        Beyaz threshold değeri (0-255, varsayılan 240)

    Returns:
    --------
    result : numpy.ndarray
        RGBA görüntü (4 kanal)
    """

    # Görüntüyü yükle
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Görüntü yüklenemedi: {image_path}")

    # BGR'den RGB'ye çevir
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width = image_rgb.shape[:2]

    # RGBA oluştur (Alpha kanalı ekle)
    result = np.zeros((height, width, 4), dtype=np.uint8)
    result[:, :, :3] = image_rgb  # RGB kanalları kopyala
    result[:, :, 3] = 255  # Alpha kanalını başta opak yap

    # Beyaz pikselleri bul
    # Tüm RGB değerleri threshold'dan büyükse beyaz say
    r, g, b = image_rgb[:,:,0], image_rgb[:,:,1], image_rgb[:,:,2]
    white_mask = (r >= white_threshold) & (g >= white_threshold) & (b >= white_threshold)

    # Beyaz piksellerin alpha değerini 0 yap (şeffaf)
    result[white_mask, 3] = 0

    # İstatistik
    total_pixels = height * width
    white_pixels = np.sum(white_mask)
    transparent_pixels = white_pixels

    print(f"Toplam piksel: {total_pixels:,}")
    print(f"Beyaz piksel: {white_pixels:,} ({white_pixels/total_pixels*100:.1f}%)")
    print(f"Şeffaf yapılan: {transparent_pixels:,}")

    # PNG olarak kaydet (şeffaflık için)
    if output_path:
        # Dosya uzantısını .png yap
        if not output_path.lower().endswith('.png'):
            output_path = os.path.splitext(output_path)[0] + '.png'

        # Klasör oluştur
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # PIL ile kaydet (alpha kanalı için)
        pil_image = Image.fromarray(result, 'RGBA')
        pil_image.save(output_path)
        print(f"Şeffaf görüntü kaydedildi: {output_path}")

    return result

def batch_make_transparent(input_folder, output_folder, white_threshold=240):
    """
    Klasördeki tüm görüntülerin beyazlarını şeffaf yapar
    """

    supported_formats = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp', '.tif']
    Path(output_folder).mkdir(parents=True, exist_ok=True)

    processed_count = 0
    error_count = 0

    print(f"Beyazları şeffaf yapma işlemi başlıyor...")
    print(f"Threshold: {white_threshold}")
    print(f"Girdi: {input_folder}")
    print(f"Çıktı: {output_folder}")
    print("-" * 50)

    for filename in os.listdir(input_folder):
        if any(filename.lower().endswith(fmt) for fmt in supported_formats):
            input_path = os.path.join(input_folder, filename)

            # Çıkış dosyası PNG olacak
            name = os.path.splitext(filename)[0]
            output_filename = f"{name}.png"
            output_path = os.path.join(output_folder, output_filename)

            try:
                print(f"İşleniyor: {filename}")
                result = make_white_transparent(input_path, output_path, white_threshold)
                processed_count += 1
                print(f"✓ {filename} -> {output_filename}")
                print()

            except Exception as e:
                error_count += 1
                print(f"✗ HATA - {filename}: {str(e)}")
                print()

    print("-" * 50)
    print(f"Tamamlandı! İşlenen: {processed_count}, Hata: {error_count}")

def preview_transparency(image_path, white_threshold=240):
    """
    Şeffaflık işlemini önizle
    """
    import matplotlib.pyplot as plt

    # Orijinal görüntü
    original = cv2.imread(image_path)
    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

    # Şeffaf yapılmış versiyon
    transparent = make_white_transparent(image_path, white_threshold=white_threshold)

    # Beyaz mask'i göster
    r, g, b = original_rgb[:,:,0], original_rgb[:,:,1], original_rgb[:,:,2]
    white_mask = (r >= white_threshold) & (g >= white_threshold) & (b >= white_threshold)

    # Görselleştir
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(original_rgb)
    axes[0].set_title('Orijinal')
    axes[0].axis('off')

    axes[1].imshow(white_mask, cmap='gray')
    axes[1].set_title(f'Beyaz Mask (threshold={white_threshold})')
    axes[1].axis('off')

    # Şeffaf görüntüyü göster (şeffaf alanlar siyah görünecek)
    display_image = transparent[:,:,:3].copy()
    display_image[transparent[:,:,3] == 0] = [0, 0, 0]  # Şeffaf alanları siyah göster

    axes[2].imshow(display_image)
    axes[2].set_title('Sonuç (şeffaf=siyah)')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

# KULLANIM ÖRNEKLERİ
if __name__ == "__main__":

    print("=== BEYAZ PİKSELLERİ ŞEFFAF YAPMA ===")
    print()

    print("1. TEK GÖRÜNTÜ:")
    print("result = make_white_transparent('image.jpg', 'output.png', white_threshold=240)")
    print()

    print("2. ÖNİZLEME:")
    print("preview_transparency('image.jpg', white_threshold=240)")
    print()

    print("3. TOPLU İŞLEM:")
    batch_make_transparent('/content/drive/MyDrive/dataset/test/images', '/content/drive/MyDrive/dataset/test/k_images', white_threshold=200)
    print()

    print("NOT: Çıkış dosyaları PNG formatında olacak (şeffaflık için)")
    print("Threshold değerini ayarlayabilirsin: 240 (varsayılan), 220 (daha az beyaz), 250 (sadece çok beyaz)")

    # Örnek kullanım (uncomment et):
    # preview_transparency('your_image.jpg', white_threshold=240)
    # result = make_white_transparent('your_image.jpg', 'output.png')
    # batch_make_transparent('input_folder/', 'output_folder/')

=== BEYAZ PİKSELLERİ ŞEFFAF YAPMA ===

1. TEK GÖRÜNTÜ:
result = make_white_transparent('image.jpg', 'output.png', white_threshold=240)

2. ÖNİZLEME:
preview_transparency('image.jpg', white_threshold=240)

3. TOPLU İŞLEM:
Beyazları şeffaf yapma işlemi başlıyor...
Threshold: 200
Girdi: /content/drive/MyDrive/dataset/test/images
Çıktı: /content/drive/MyDrive/dataset/test/k_images
--------------------------------------------------
İşleniyor: ZT80_38_C_5_4.jpg
Toplam piksel: 9,610,000
Beyaz piksel: 4,479,408 (46.6%)
Şeffaf yapılan: 4,479,408
Şeffaf görüntü kaydedildi: /content/drive/MyDrive/dataset/test/k_images/ZT80_38_C_5_4.png
✓ ZT80_38_C_5_4.jpg -> ZT80_38_C_5_4.png

İşleniyor: ZT80_38_C_5_6.jpg
Toplam piksel: 9,610,000
Beyaz piksel: 4,090,152 (42.6%)
Şeffaf yapılan: 4,090,152
Şeffaf görüntü kaydedildi: /content/drive/MyDrive/dataset/test/k_images/ZT80_38_C_5_6.png
✓ ZT80_38_C_5_6.jpg -> ZT80_38_C_5_6.png

İşleniyor: ZT80_38_C_5_7.jpg
Toplam piksel: 9,610,000
Beyaz piksel: 3,458,555 (36

In [ ]:
import os

def compare_folders(images_folder, k_images_folder):
    """
    images (jpg) ve k_images (png) klasörlerini karşılaştırır
    k_images'te eksik olanları listeler
    """

    print("📁 KLASÖR KARŞILAŞTIRMA")
    print("=" * 50)

    # images klasöründeki JPG dosyaları al
    jpg_files = []
    for file in os.listdir(images_folder):
        if file.lower().endswith(('.jpg', '.jpeg')):
            # Sadece dosya adı (uzantısız)
            name_without_ext = os.path.splitext(file)[0]
            jpg_files.append(name_without_ext)

    print(f"📷 images klasöründe JPG dosya sayısı: {len(jpg_files)}")

    # k_images klasöründeki PNG dosyaları al
    png_files = []
    for file in os.listdir(k_images_folder):
        if file.lower().endswith('.png'):
            # Sadece dosya adı (uzantısız)
            name_without_ext = os.path.splitext(file)[0]
            png_files.append(name_without_ext)

    print(f"🖼️  k_images klasöründe PNG dosya sayısı: {len(png_files)}")

    # Set'e çevir (hızlı karşılaştırma için)
    jpg_set = set(jpg_files)
    png_set = set(png_files)

    # Eksik dosyaları bul
    missing_in_k_images = jpg_set - png_set
    extra_in_k_images = png_set - jpg_set

    print("\n" + "=" * 50)
    print("📊 KARŞILAŞTIRMA SONUCU")
    print("=" * 50)

    print(f"❌ k_images'te EKSIK olan dosyalar: {len(missing_in_k_images)}")
    if missing_in_k_images:
        print("\nEksik dosyalar:")
        for i, file in enumerate(sorted(missing_in_k_images), 1):
            print(f"   {i}. {file}.jpg -> {file}.png (eksik)")

    print(f"\n➕ k_images'te FAZLA olan dosyalar: {len(extra_in_k_images)}")
    if extra_in_k_images:
        print("\nFazla dosyalar:")
        for i, file in enumerate(sorted(extra_in_k_images), 1):
            print(f"   {i}. {file}.png (fazla)")

    # Eşleşenler
    matching = jpg_set & png_set
    print(f"\n✅ Eşleşen dosyalar: {len(matching)}")

    print("\n" + "=" * 50)
    print("ÖZET:")
    print(f"JPG toplam: {len(jpg_files)}")
    print(f"PNG toplam: {len(png_files)}")
    print(f"Eşleşen: {len(matching)}")
    print(f"Eksik: {len(missing_in_k_images)}")
    print(f"Fazla: {len(extra_in_k_images)}")

    return {
        'jpg_files': jpg_files,
        'png_files': png_files,
        'missing_in_k_images': missing_in_k_images,
        'extra_in_k_images': extra_in_k_images,
        'matching': matching
    }

def create_missing_list(images_folder, k_images_folder, output_file="missing_files.txt"):
    """
    Eksik dosyaları txt dosyasına yazar
    """
    result = compare_folders(images_folder, k_images_folder)

    # Eksik dosyaları txt'ye yaz
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("K_IMAGES KLASÖRÜNDE EKSİK OLAN DOSYALAR\n")
        f.write("=" * 50 + "\n\n")

        for i, file in enumerate(sorted(result['missing_in_k_images']), 1):
            f.write(f"{i}. {file}.jpg -> {file}.png\n")

        f.write(f"\nToplam eksik: {len(result['missing_in_k_images'])}\n")

    print(f"\n💾 Eksik dosya listesi kaydedildi: {output_file}")

if __name__ == "__main__":

    # BURALARI DEĞİŞTİR - KENDİ KLASÖR YOLLARIN
    images_folder = "/content/drive/MyDrive/dataset/train/images"          # JPG dosyalarının olduğu klasör
    k_images_folder = "/content/drive/MyDrive/dataset/train/k_images"      # PNG dosyalarının olduğu klasör

    print("🚀 Eksik dosya bulma başlıyor...")
    print(f"📁 images klasörü: {images_folder}")
    print(f"📁 k_images klasörü: {k_images_folder}")
    print()

    # Karşılaştır
    result = compare_folders(images_folder, k_images_folder)

    # Eksik dosyaları txt'ye kaydet
    create_missing_list(images_folder, k_images_folder, "eksik_dosyalar.txt")

    print("\n✅ İşlem tamamlandı!")

🚀 Eksik dosya bulma başlıyor...
📁 images klasörü: /content/drive/MyDrive/dataset/train/images
📁 k_images klasörü: /content/drive/MyDrive/dataset/train/k_images

📁 KLASÖR KARŞILAŞTIRMA
📷 images klasöründe JPG dosya sayısı: 641
🖼️  k_images klasöründe PNG dosya sayısı: 638

📊 KARŞILAŞTIRMA SONUCU
❌ k_images'te EKSIK olan dosyalar: 3

Eksik dosyalar:
   1. ZT204_6_B_7_4.jpg -> ZT204_6_B_7_4.png (eksik)
   2. ZT204_6_B_7_8.jpg -> ZT204_6_B_7_8.png (eksik)
   3. ZT204_6_B_8_1.jpg -> ZT204_6_B_8_1.png (eksik)

➕ k_images'te FAZLA olan dosyalar: 0

✅ Eşleşen dosyalar: 638

ÖZET:
JPG toplam: 641
PNG toplam: 638
Eşleşen: 638
Eksik: 3
Fazla: 0
📁 KLASÖR KARŞILAŞTIRMA
📷 images klasöründe JPG dosya sayısı: 641
🖼️  k_images klasöründe PNG dosya sayısı: 638

📊 KARŞILAŞTIRMA SONUCU
❌ k_images'te EKSIK olan dosyalar: 3

Eksik dosyalar:
   1. ZT204_6_B_7_4.jpg -> ZT204_6_B_7_4.png (eksik)
   2. ZT204_6_B_7_8.jpg -> ZT204_6_B_7_8.png (eksik)
   3. ZT204_6_B_8_1.jpg -> ZT204_6_B_8_1.png (eksik)

➕ k_image